In [279]:
import requests
from lxml import html
import pandas as pd


In [280]:
def make_dataframe(response):
    parsed_body = html.fromstring(response.text, parser = html.HTMLParser(encoding='utf-8',remove_blank_text=True))

    prices1 =[pr.text.strip().replace("руб "," ") for pr in  parsed_body.xpath('//div[@class="about"]')]
    prices =[price.replace(" руб.","").replace(" ","") for price in prices1]

    print("Найдено Элементов",len(prices))
    addresses = [ad.strip() for ad in parsed_body.xpath('//p[@class="address fader"]/text()')]


    links = parsed_body.xpath('//a[@class="item-description-title-link"]/@href')
    names = [name.strip() for name in parsed_body.xpath('//a[@class="item-description-title-link"]/text()')]
    square_meters = [a.split(" ")[2] for a in names]
    floors = [a.split(" ")[4].split('/')[0] for a in names]
    total_floors=[a.split(" ")[4].split('/')[1] for a in names]
    rooms=[a[0] for a in names]
    df = pd.DataFrame({ #'Name' :  pd.Series([name for name in names]),
                     'Address' : pd.Series([name for name in addresses]),
                     'Price' :  pd.Series([name for name in prices]),
                     'Link' : pd.Series(['https://www.avito.ru'+name for name in links]),
                       'Square_meters':pd.Series(square_meters),
                       'Floor':pd.Series(floors),
                       'Total Floors':pd.Series(total_floors),
                       'Rooms':pd.Series(rooms),
                   })
    return df


In [281]:
i = 0
r = 'https://www.avito.ru/saratov/kvartiry/prodam/3-komnatnye?p='+str(i)+'&district=28'
response = requests.get(r)
while (response.status_code==200):
    df = make_dataframe(response)
    df.to_csv('dataset.csv', mode='a', header=False,sep=';',index=False)
    print(i)
    i=i+1
    r = 'https://www.avito.ru/saratov/kvartiry/prodam/3-komnatnye?p='+str(i)+'&district=28'
    response = requests.get(r)


Найдено Элементов 50


PermissionError: [Errno 13] Permission denied: 'dataset.csv'